In [1]:
import unicodedata
import re
import numpy as np
import os
import io
import time

# Reading Data 

# Spanish - to - english

In [2]:
path =  "data/spa.txt"

In [3]:
file = io.open(path,encoding = "UTF-8").read()

In [4]:
lines = file.split("\n")

In [5]:
[[w for w in l.split("\t")[:2]] for l in lines[0:3]]

[['Go.', 'Ve.'], ['Go.', 'Vete.'], ['Go.', 'Vaya.']]

In [12]:
import tensorflow as tf

In [13]:
tf.__version__

'2.1.0'

In [15]:
tf.keras.preprocessing.sequence.pad_sequences(
                            [[1,1,2,33,3],
                             [1,1,2,33,3,1,1,1]]
                             ,padding='post')

array([[ 1,  1,  2, 33,  3,  0,  0,  0],
       [ 1,  1,  2, 33,  3,  1,  1,  1]])

# Hindi - English

In [7]:
path =  "data/hin.txt"
file = io.open(path,encoding = "UTF-8").read()
lines = file.split("\n")
words = [[w for w in l.split("\t")[:2]] for l in lines[0:100]]

In [16]:
words[10][1].split()

['मैं', 'ठीक', 'हूँ।']

In [26]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    
      return ''.join(c for c in unicodedata.normalize('NFD', s)
          if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w,):

      w = unicode_to_ascii(w.lower().strip())

      
      w = re.sub(r"([?.!,¿])", r" \1 ", w)
      w = re.sub(r'[" "]+', " ", w)

      # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
      w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

      w = w.strip()

      # adding a start and an end token to the sentence
      # so that the model know when to start and stop predicting.
      w = '<start> ' + w + ' <end>'
      return w




In [29]:

for tar,inp in words[0:10]:
    
    print('input',' '.join([preprocess_sentence(w) for w in inp.split()]))

    print('target:',' '.join([preprocess_sentence(w) for w in tar.split()]))
    


input <start> ! <end>
target: <start> wow ! <end>
input <start> ! <end>
target: <start> help ! <end>
input <start> . <end>
target: <start> jump . <end>
input <start> . <end>
target: <start> jump . <end>
input <start> . <end>
target: <start> jump . <end>
input <start>  <end>
target: <start> hello ! <end>
input <start>  <end>
target: <start> hello ! <end>
input <start> ! <end>
target: <start> cheers ! <end>
input <start> ! <end>
target: <start> cheers ! <end>
input <start>  <end> <start>  <end> <start> ? <end>
target: <start> got <end> <start> it ? <end>


# Will need to chnage preprocess function to suport hindi

In [36]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    
      return ''.join(c for c in unicodedata.normalize('NFD', s)
          if unicodedata.category(c) != 'Mn')


def preprocess_sentence(w,regex = True):

      w = unicode_to_ascii(w.lower().strip())

      if regex:
    
          w = re.sub(r"([?.!,¿])", r" \1 ", w)
          w = re.sub(r'[" "]+', " ", w)


          # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
          w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

      w = w.strip()

      # adding a start and an end token to the sentence
      # so that the model know when to start and stop predicting.
      w = '<start> ' + w + ' <end>'
      return w


for tar,inp in words[0:10]:
    
    print('input',' '.join([preprocess_sentence(inp,regex = False)]))

    print('target:',' '.join([preprocess_sentence(tar)]))
    
    


input <start> वाह! <end>
target: <start> wow ! <end>
input <start> बचाओ! <end>
target: <start> help ! <end>
input <start> उछलो. <end>
target: <start> jump . <end>
input <start> कदो. <end>
target: <start> jump . <end>
input <start> छलाग. <end>
target: <start> jump . <end>
input <start> नमसत। <end>
target: <start> hello ! <end>
input <start> नमसकार। <end>
target: <start> hello ! <end>
input <start> वाह-वाह! <end>
target: <start> cheers ! <end>
input <start> चियरस! <end>
target: <start> cheers ! <end>
input <start> समझ कि नही? <end>
target: <start> got it ? <end>


In [64]:
# perfect match 

# ref =  ['i', 'm', 'very', 'optimistic', '.']
# mt  = ['i', 'm', 'very', 'optimistic', '.']

reference =   ['you', 'need', 'to', 'shut', 'up', '.']
mt  =  ['you', 'must', 'be', 'punished', '.']


# can have more than 1 ref 

from collections import Counter

def get_counts(x):

  return Counter(x)


def get_striped_counts(mt,ref):

  # check if the translated word is present in the refrence word.

  '''
  number of t

  '''

  mt_dict = get_counts(mt)

  ref_dict = get_counts(ref)    
      
     
    

  #ref_dict = get_counts(ref)
  # common words between ref and mt 

  num = 0
  den = 0


  for words in mt_dict.keys():
      #number of times mt_words occur in ref_words
        
         print(f' no. of times \'{words}\' occur in  ref sent is \'{ref_dict[words]}\'')
     
         num += ref_dict[words]

         den += mt_dict[words]
  '''
  calculate : no. of times word occurs in ref / no. of times word occurs in mt only
  '''
  return num/den 

# get_counts(mt).keys()
get_striped_counts(mt,ref)

 no. of times 'you' occur in  ref sent is '1'
 no. of times 'must' occur in  ref sent is '0'
 no. of times 'be' occur in  ref sent is '0'
 no. of times 'punished' occur in  ref sent is '0'
 no. of times '.' occur in  ref sent is '1'


0.4

In [65]:
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'a', 'test'],['this','is','test']]
candidate = ['this', 'is', 'a','test']
score = sentence_bleu(reference, candidate,weights = (1,0,0,0))
print(score)

1.0


In [67]:
get_striped_counts(candidate,reference[0])

 no. of times 'this' occur in  ref sent is '1'
 no. of times 'is' occur in  ref sent is '1'
 no. of times 'a' occur in  ref sent is '1'
 no. of times 'test' occur in  ref sent is '1'


1.0